## [SQL Model](https://sqlmodel.tiangolo.com/)

* [Tutorial](https://sqlmodel.tiangolo.com/tutorial/)

In [1]:
from typing import Optional, List

from sqlmodel import Field, SQLModel, create_engine, Session, select, Relationship

In [2]:
# for formatting and color

from pprint import pprint as pp
from pprint import pformat as pf
from termcolor import cprint
from colorama import Fore, Back, Style

In [3]:
sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=True)


def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

## [Creating JOIN](https://sqlmodel.tiangolo.com/tutorial/connect/create-connected-tables/)

In [4]:
class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    headquarters: str


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

    team_id: Optional[int] = Field(default=None, foreign_key="team.id")

In [5]:
create_db_and_tables()

2021-09-08 17:13:57,624 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:13:57,626 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("team")
2021-09-08 17:13:57,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:13:57,631 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("team")
2021-09-08 17:13:57,631 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:13:57,632 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2021-09-08 17:13:57,633 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:13:57,634 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2021-09-08 17:13:57,635 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:13:57,636 INFO sqlalchemy.engine.Engine 
CREATE TABLE team (
	id INTEGER, 
	name VARCHAR NOT NULL, 
	headquarters VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2021-09-08 17:13:57,637 INFO sqlalchemy.engine.Engine [no key 0.00074s] ()
2021-09-08 17:13:57,785 INFO sqlalchemy.engine.Engine CREATE INDEX ix_team_na

### [Creating Connected Rows](https://sqlmodel.tiangolo.com/tutorial/connect/create-connected-rows/)

In [6]:
def create_heroes():
    with Session(engine) as session:
        # creating teams
        team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
        team_z_force = Team(name="Z-Force", headquarters="Sister Margaret’s Bar")
        session.add(team_preventers)
        session.add(team_z_force)
        session.commit()

        # creating heroes and assigning them to team
        hero_deadpond = Hero(
            name="Deadpond", secret_name="Dive Wilson", team_id=team_z_force.id
        )
        hero_rusty_man = Hero(
            name="Rusty-Man",
            secret_name="Tommy Sharp",
            age=48,
            team_id=team_preventers.id,
        )
        hero_spider_boy = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
        session.add(hero_deadpond)
        session.add(hero_rusty_man)
        session.add(hero_spider_boy)
        session.commit()

        # refresh objects and print them
        session.refresh(hero_deadpond)
        session.refresh(hero_rusty_man)
        session.refresh(hero_spider_boy)

        cprint(f"Created hero: {hero_deadpond}", "green")
        cprint(f"Created hero: {hero_rusty_man}", "green")
        cprint(f"Created hero: {hero_spider_boy}", "green")

In [7]:
create_heroes()

2021-09-08 17:14:03,013 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:03,017 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:14:03,018 INFO sqlalchemy.engine.Engine [generated in 0.00135s] ('Preventers', 'Sharp Tower')
2021-09-08 17:14:03,023 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:14:03,024 INFO sqlalchemy.engine.Engine [cached since 0.006726s ago] ('Z-Force', 'Sister Margaret’s Bar')
2021-09-08 17:14:03,025 INFO sqlalchemy.engine.Engine COMMIT
2021-09-08 17:14:03,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:03,243 INFO sqlalchemy.engine.Engine SELECT team.id AS team_id, team.name AS team_name, team.headquarters AS team_headquarters 
FROM team 
WHERE team.id = ?
2021-09-08 17:14:03,246 INFO sqlalchemy.engine.Engine [generated in 0.00384s] (2,)
2021-09-08 17:14:03,253 INFO sqlalchemy.engine.Engine SELECT team.id AS team_id, team.name AS team_n

### [Reading connected data](https://sqlmodel.tiangolo.com/tutorial/connect/read-connected-data/)

In [8]:
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero, Team).where(Hero.team_id == Team.id)
        results = session.exec(statement)
        for hero, team in results:
            cprint(f"Hero: {hero} \n\tTeam: {team}\n", "green")

In [9]:
select_heroes()

2021-09-08 17:14:08,102 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:08,103 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero, team 
WHERE hero.team_id = team.id
2021-09-08 17:14:08,104 INFO sqlalchemy.engine.Engine [no key 0.00080s] ()
Hero: secret_name='Dive Wilson' name='Deadpond' team_id=2 id=1 age=None 
	Team: name='Z-Force' headquarters='Sister Margaret’s Bar' id=2

Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 id=2 age=48 
	Team: name='Preventers' headquarters='Sharp Tower' id=1

2021-09-08 17:14:08,105 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
# joining tables, this is equivalent to above
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero, Team).join(Team)
        results = session.exec(statement)
        for hero, team in results:
            cprint(f"Hero: {hero} \n\tTeam: {team}\n", "green")


select_heroes()

2021-09-08 17:14:12,179 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:12,182 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero JOIN team ON team.id = hero.team_id
2021-09-08 17:14:12,183 INFO sqlalchemy.engine.Engine [no key 0.00107s] ()
Hero: secret_name='Dive Wilson' name='Deadpond' team_id=2 id=1 age=None 
	Team: name='Z-Force' headquarters='Sister Margaret’s Bar' id=2

Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 id=2 age=48 
	Team: name='Preventers' headquarters='Sharp Tower' id=1

2021-09-08 17:14:12,185 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
# left outer join
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero, Team).join(Team, isouter=True)
        results = session.exec(statement)
        for hero, team in results:
            cprint(f"Hero: {hero} \n\tTeam: {team}\n", "green")


select_heroes()

2021-09-08 17:14:15,846 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:15,847 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero LEFT OUTER JOIN team ON team.id = hero.team_id
2021-09-08 17:14:15,848 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
Hero: secret_name='Dive Wilson' name='Deadpond' team_id=2 id=1 age=None 
	Team: name='Z-Force' headquarters='Sister Margaret’s Bar' id=2

Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 id=2 age=48 
	Team: name='Preventers' headquarters='Sharp Tower' id=1

Hero: secret_name='Pedro Parqueador' name='Spider-Boy' team_id=None id=3 age=None 
	Team: None

2021-09-08 17:14:15,852 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
# Select Only Heroes But Join with Teams
# If we only put the Team in the .join() and not in the select() function, we would not get the team data.
# But we would still be able to filter the rows with it.
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero).join(Team).where(Team.name == "Preventers")
        results = session.exec(statement)
        for hero in results:
            cprint(f"Hero: {hero}", "green")


select_heroes()

2021-09-08 17:14:20,129 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:20,131 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero JOIN team ON team.id = hero.team_id 
WHERE team.name = ?
2021-09-08 17:14:20,131 INFO sqlalchemy.engine.Engine [no key 0.00073s] ('Preventers',)
Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 id=2 age=48
2021-09-08 17:14:20,133 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
# Include the Team
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero, Team).join(Team).where(Team.name == "Preventers")
        results = session.exec(statement)
        for hero, team in results:
            cprint(f"Hero: {hero} \n\tTeam: {team}\n", "green")


select_heroes()

2021-09-08 17:14:22,863 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:22,865 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero JOIN team ON team.id = hero.team_id 
WHERE team.name = ?
2021-09-08 17:14:22,865 INFO sqlalchemy.engine.Engine [no key 0.00067s] ('Preventers',)
Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 id=2 age=48 
	Team: name='Preventers' headquarters='Sharp Tower' id=1

2021-09-08 17:14:22,867 INFO sqlalchemy.engine.Engine ROLLBACK


### [Update Data Connections](https://sqlmodel.tiangolo.com/tutorial/connect/update-data-connections/)

In [14]:
def update_spider_boy():
    with Session(engine) as session:
        # fetch hero and team
        hero_spider_boy = session.exec(
            select(Hero).where(Hero.name == "Spider-Boy")
        ).one()
        team_preventers = session.exec(
            select(Team).where(Team.name == "Preventers")
        ).one()
        cprint(f"Hero: {hero_spider_boy} \n\tTeam: {team_preventers}", "green")

        # update connection
        hero_spider_boy.team_id = team_preventers.id
        session.add(hero_spider_boy)
        # commit to db, update and print
        session.commit()
        session.refresh(hero_spider_boy)
        cprint(f"Updated Hero: {hero_spider_boy}", "green")


update_spider_boy()

2021-09-08 17:14:27,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:27,397 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.name = ?
2021-09-08 17:14:27,398 INFO sqlalchemy.engine.Engine [no key 0.00112s] ('Spider-Boy',)
2021-09-08 17:14:27,400 INFO sqlalchemy.engine.Engine SELECT team.id, team.name, team.headquarters 
FROM team 
WHERE team.name = ?
2021-09-08 17:14:27,401 INFO sqlalchemy.engine.Engine [no key 0.00061s] ('Preventers',)
Hero: secret_name='Pedro Parqueador' name='Spider-Boy' team_id=None id=3 age=None 
	Team: name='Preventers' headquarters='Sharp Tower' id=1
2021-09-08 17:14:27,403 INFO sqlalchemy.engine.Engine UPDATE hero SET team_id=? WHERE hero.id = ?
2021-09-08 17:14:27,403 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1, 3)
2021-09-08 17:14:27,406 INFO sqlalchemy.engine.Engine COMMIT
2021-09-08 17:14:27,905 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17

### [Removing Data Connections](https://sqlmodel.tiangolo.com/tutorial/connect/remove-data-connections/)

In [15]:
def update_spider_boy():
    with Session(engine) as session:
        # fetch hero and team
        hero_spider_boy = session.exec(
            select(Hero).where(Hero.name == "Spider-Boy")
        ).one()
        cprint(f"Hero: {hero_spider_boy}", "green")

        # update connection
        hero_spider_boy.team_id = None
        session.add(hero_spider_boy)
        # commit to db, update and print
        session.commit()
        session.refresh(hero_spider_boy)
        cprint(f"Updated Hero: {hero_spider_boy}", "green")


update_spider_boy()

2021-09-08 17:14:32,070 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:32,072 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.name = ?
2021-09-08 17:14:32,073 INFO sqlalchemy.engine.Engine [no key 0.00079s] ('Spider-Boy',)
Hero: secret_name='Pedro Parqueador' name='Spider-Boy' team_id=1 id=3 age=None
2021-09-08 17:14:32,075 INFO sqlalchemy.engine.Engine UPDATE hero SET team_id=? WHERE hero.id = ?
2021-09-08 17:14:32,076 INFO sqlalchemy.engine.Engine [cached since 4.673s ago] (None, 3)
2021-09-08 17:14:32,080 INFO sqlalchemy.engine.Engine COMMIT
2021-09-08 17:14:32,288 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:14:32,289 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.id = ?
2021-09-08 17:14:32,289 INFO sqlalchemy.engine.Engine [cached since 28.87s ago] (3,)
Updated Hero: secret_name='Pedro Parqueador' name='

## [Relationships](https://sqlmodel.tiangolo.com/tutorial/relationship-attributes/define-relationships-attributes/)

In [4]:
# RESTART KERNEL, RE-IMPORT AND REMOVE DATABASE.DB
from sqlmodel import Relationship

# READ FOR CLARIFICATION - https://sqlmodel.tiangolo.com/tutorial/relationship-attributes/back-populates/
# SQLModel.metadata.drop_all(engine)
# SQLModel.metadata.clear()


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    headquarters: str

    heroes: List["Hero"] = Relationship(back_populates="team")


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

    team_id: Optional[int] = Field(default=None, foreign_key="team.id")
    team: Optional[Team] = Relationship(back_populates="heroes")


create_db_and_tables()

2021-09-08 17:15:03,554 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:15:03,556 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("team")
2021-09-08 17:15:03,557 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:15:03,558 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("team")
2021-09-08 17:15:03,559 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:15:03,559 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2021-09-08 17:15:03,560 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:15:03,560 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2021-09-08 17:15:03,561 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:15:03,562 INFO sqlalchemy.engine.Engine 
CREATE TABLE team (
	id INTEGER, 
	name VARCHAR NOT NULL, 
	headquarters VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2021-09-08 17:15:03,562 INFO sqlalchemy.engine.Engine [no key 0.00044s] ()
2021-09-08 17:15:03,635 INFO sqlalchemy.engine.Engine CREATE INDEX ix_team_na

### [Create/Update Relationships](https://sqlmodel.tiangolo.com/tutorial/relationship-attributes/create-and-update-relationships/)

In [5]:
def create_heroes():
    with Session(engine) as session:
        team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
        team_z_force = Team(name="Z-Force", headquarters="Sister Margaret’s Bar")

        hero_deadpond = Hero(
            name="Deadpond", secret_name="Dive Wilson", team=team_z_force
        )
        hero_rusty_man = Hero(
            name="Rusty-Man", secret_name="Tommy Sharp", age=48, team=team_preventers
        )
        hero_spider_boy = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
        session.add(hero_deadpond)
        session.add(hero_rusty_man)
        session.add(hero_spider_boy)
        session.commit()

        session.refresh(hero_deadpond)
        session.refresh(hero_rusty_man)
        session.refresh(hero_spider_boy)

        cprint(f"Created hero: {hero_deadpond}", "green")
        cprint(f"Created hero: {hero_rusty_man}", "green")
        cprint(f"Created hero: {hero_spider_boy}", "green")

        hero_spider_boy.team = team_preventers
        session.add(hero_spider_boy)
        session.commit()


create_heroes()

2021-09-08 17:15:07,026 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:15:07,027 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:15:07,028 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ('Z-Force', 'Sister Margaret’s Bar')
2021-09-08 17:15:07,130 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:15:07,131 INFO sqlalchemy.engine.Engine [cached since 0.1037s ago] ('Preventers', 'Sharp Tower')
2021-09-08 17:15:07,132 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age, team_id) VALUES (?, ?, ?, ?)
2021-09-08 17:15:07,132 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ('Deadpond', 'Dive Wilson', None, 1)
2021-09-08 17:15:07,133 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age, team_id) VALUES (?, ?, ?, ?)
2021-09-08 17:15:07,134 INFO sqlalchemy.engine.Engine [cached since 0.002244s ago] ('Rusty-Man', 'Tommy Sharp', 48, 2)
2021-09

There are several things to notice here.

First, we create some Team instance objects. We want to use the IDs of these teams when creating the Hero instances, in the team_id field.

But model instances don't have an ID generated by the database until we add and commit them to the session. Before that, they are just None, and we want to use the actual IDs.

So, we have to add them and commit the session first, before we start creating the Hero instances, to be able to use their IDs.

Then, we use those IDs when creating the Hero instances. We add the new heroes to the session, and then we commit them.

So, we are committing twice. And we have to remember to add some things first, and then commit, and do all that in the right order, otherwise we could end up using a team.id that is currently None because it hasn't been saved.

This is the first area where these relationship attributes can help. 


In [9]:
with Session(engine) as session:
    session.execute("delete from hero;")
    session.execute("delete from team;")
    session.commit()

2021-09-08 17:21:59,792 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:21:59,793 INFO sqlalchemy.engine.Engine delete from hero;
2021-09-08 17:21:59,793 INFO sqlalchemy.engine.Engine [generated in 0.00040s] ()
2021-09-08 17:21:59,797 INFO sqlalchemy.engine.Engine delete from team;
2021-09-08 17:21:59,797 INFO sqlalchemy.engine.Engine [generated in 0.00043s] ()
2021-09-08 17:21:59,798 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
def create_heroes():
    with Session(engine) as session:
        # create teams
        team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
        team_z_force = Team(name="Z-Force", headquarters="Sister Margaret’s Bar")

        # create heroes
        hero_deadpond = Hero(
            name="Deadpond", secret_name="Dive Wilson", team=team_z_force
        )
        hero_rusty_man = Hero(
            name="Rusty-Man", secret_name="Tommy Sharp", age=48, team=team_preventers
        )
        hero_spider_boy = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
        session.add(hero_deadpond)
        session.add(hero_rusty_man)
        session.add(hero_spider_boy)
        session.commit()

        # refresh objects
        session.refresh(hero_deadpond)
        session.refresh(hero_rusty_man)
        session.refresh(hero_spider_boy)

        cprint(f"Created hero: {hero_deadpond}", "green")
        cprint(f"Created hero: {hero_rusty_man}", "green")
        cprint(f"Created hero: {hero_spider_boy}", "green")

        # now we can update team without explictly specifying team.id
        hero_spider_boy.team = team_preventers
        session.add(hero_spider_boy)
        session.commit()
        session.refresh(hero_spider_boy)
        cprint(f"Updated hero: {hero_spider_boy}", "green")

        # creating a team with heroes
        hero_black_lion = Hero(name="Black Lion", secret_name="Trevor Challa", age=35)
        hero_sure_e = Hero(name="Princess Sure-E", secret_name="Sure-E")
        team_wakaland = Team(
            name="Wakaland",
            headquarters="Wakaland Capital City",
            heroes=[hero_black_lion, hero_sure_e],
        )
        session.add(team_wakaland)
        session.commit()
        session.refresh(team_wakaland)
        cprint(pf(f"Team Wakaland: {team_wakaland}"), "green")

        # include relationship object in many side
        hero_tarantula = Hero(name="Tarantula", secret_name="Natalia Roman-on", age=32)
        hero_dr_weird = Hero(name="Dr. Weird", secret_name="Steve Weird", age=36)
        hero_cap = Hero(
            name="Captain North America", secret_name="Esteban Rogelios", age=93
        )

        team_preventers.heroes.append(hero_tarantula)
        team_preventers.heroes.append(hero_dr_weird)
        team_preventers.heroes.append(hero_cap)
        session.add(team_preventers)
        session.commit()
        session.refresh(hero_tarantula)
        session.refresh(hero_dr_weird)
        session.refresh(hero_cap)
        cprint(f"Preventers new hero: {hero_tarantula}", "green")
        cprint(f"Preventers new hero: {hero_dr_weird}", "green")
        cprint(f"Preventers new hero: {hero_cap}", "green")


create_heroes()

2021-09-08 17:22:02,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:22:02,964 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:22:02,966 INFO sqlalchemy.engine.Engine [cached since 415.9s ago] ('Z-Force', 'Sister Margaret’s Bar')
2021-09-08 17:22:02,981 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:22:02,982 INFO sqlalchemy.engine.Engine [cached since 416s ago] ('Preventers', 'Sharp Tower')
2021-09-08 17:22:02,983 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age, team_id) VALUES (?, ?, ?, ?)
2021-09-08 17:22:02,984 INFO sqlalchemy.engine.Engine [cached since 415.9s ago] ('Deadpond', 'Dive Wilson', None, 1)
2021-09-08 17:22:02,986 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age, team_id) VALUES (?, ?, ?, ?)
2021-09-08 17:22:02,986 INFO sqlalchemy.engine.Engine [cached since 415.9s ago] ('Rusty-Man', 'Tommy Sharp', 48, 2)
2021-09-0

### [Reading Relationships](https://sqlmodel.tiangolo.com/tutorial/relationship-attributes/read-relationships/)

In [11]:
# easier way to fetch hero's team
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero).where(Hero.name == "Spider-Boy")
        result = session.exec(statement)
        hero_spider_boy = result.one()

        cprint(f"Spider-Boy's team again: {hero_spider_boy.team}", "green")


select_heroes()

2021-09-08 17:22:06,973 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:22:06,975 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.name = ?
2021-09-08 17:22:06,976 INFO sqlalchemy.engine.Engine [no key 0.00099s] ('Spider-Boy',)
2021-09-08 17:22:06,978 INFO sqlalchemy.engine.Engine SELECT team.id AS team_id, team.name AS team_name, team.headquarters AS team_headquarters 
FROM team 
WHERE team.id = ?
2021-09-08 17:22:06,979 INFO sqlalchemy.engine.Engine [generated in 0.00070s] (2,)
Spider-Boy's team again: headquarters='Sharp Tower' id=2 name='Preventers'
2021-09-08 17:22:06,980 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
# all heroes of team
def select_heroes():
    with Session(engine) as session:
        statement = select(Team).where(Team.name == "Preventers")
        result = session.exec(statement)
        team_preventers = result.one()
        cprint("Preventers heroes:", "green")
        cprint(pf(team_preventers.heroes), "green")


select_heroes()

2021-09-08 17:22:09,977 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:22:09,979 INFO sqlalchemy.engine.Engine SELECT team.id, team.name, team.headquarters 
FROM team 
WHERE team.name = ?
2021-09-08 17:22:09,980 INFO sqlalchemy.engine.Engine [no key 0.00092s] ('Preventers',)
Preventers heroes:
2021-09-08 17:22:09,982 INFO sqlalchemy.engine.Engine SELECT hero.id AS hero_id, hero.name AS hero_name, hero.secret_name AS hero_secret_name, hero.age AS hero_age, hero.team_id AS hero_team_id 
FROM hero 
WHERE ? = hero.team_id
2021-09-08 17:22:09,983 INFO sqlalchemy.engine.Engine [cached since 415.2s ago] (2,)
[Hero(secret_name='Tommy Sharp', name='Rusty-Man', team_id=2, age=48, id=2),
 Hero(secret_name='Pedro Parqueador', name='Spider-Boy', team_id=2, age=None, id=3),
 Hero(secret_name='Natalia Roman-on', name='Tarantula', team_id=2, age=32, id=6),
 Hero(secret_name='Steve Weird', name='Dr. Weird', team_id=2, age=36, id=7),
 Hero(secret_name='Esteban Rogelios', name='Captain Nort

### [Removing Relationships](https://sqlmodel.tiangolo.com/tutorial/relationship-attributes/remove-relationships/)

In [13]:
# Same as previous
def update_heroes():
    with Session(engine) as session:
        statement = select(Hero).where(Hero.name == "Spider-Boy")
        result = session.exec(statement)
        hero_spider_boy = result.one()

        hero_spider_boy.team = None
        session.add(hero_spider_boy)
        session.commit()

        session.refresh(hero_spider_boy)
        cprint(f"Spider-Boy without team: {hero_spider_boy}", "green")


update_heroes()

2021-09-08 17:22:14,521 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:22:14,522 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.name = ?
2021-09-08 17:22:14,523 INFO sqlalchemy.engine.Engine [no key 0.00076s] ('Spider-Boy',)
2021-09-08 17:22:14,525 INFO sqlalchemy.engine.Engine UPDATE hero SET team_id=? WHERE hero.id = ?
2021-09-08 17:22:14,526 INFO sqlalchemy.engine.Engine [cached since 427.2s ago] (None, 3)
2021-09-08 17:22:14,529 INFO sqlalchemy.engine.Engine COMMIT
2021-09-08 17:22:14,717 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:22:14,718 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.id = ?
2021-09-08 17:22:14,718 INFO sqlalchemy.engine.Engine [cached since 427.5s ago] (3,)
Spider-Boy without team: secret_name='Pedro Parqueador' name='Spider-Boy' team_id=None age=None id=3
2021-09-08 17:22:14,720 INFO 

### Important Reading
### [Relationship Back populates](https://sqlmodel.tiangolo.com/tutorial/relationship-attributes/back-populates/)
### [Type annotation strings](https://sqlmodel.tiangolo.com/tutorial/relationship-attributes/type-annotation-strings/)

## Many to Many Relationships
### [Create Models with a Many-to-Many Link](https://sqlmodel.tiangolo.com/tutorial/many-to-many/create-models-with-link/)

In [4]:
# RESTART KERNEL, RE-IMPORT AND REMOVE DATABASE.DB
class HeroTeamLink(SQLModel, table=True):
    team_id: Optional[int] = Field(default=None, foreign_key="team.id", primary_key=True)
    hero_id: Optional[int] = Field(default=None, foreign_key="hero.id", primary_key=True)


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    headquarters: str
    # added link model to HeroTeamLink for storing many to many relationships
    heroes: List["Hero"] = Relationship(back_populates="teams", link_model=HeroTeamLink)


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None
    # added link model to HeroTeamLink for storing many to many relationships
    teams: List[Team] = Relationship(back_populates="heroes", link_model=HeroTeamLink)


create_db_and_tables()

2021-09-08 17:24:04,921 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:24:04,924 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("heroteamlink")
2021-09-08 17:24:04,925 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:24:04,927 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("heroteamlink")
2021-09-08 17:24:04,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:24:04,928 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("team")
2021-09-08 17:24:04,928 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:24:04,929 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("team")
2021-09-08 17:24:04,930 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:24:04,930 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2021-09-08 17:24:04,931 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:24:04,932 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2021-09-08 17:24:04,932 INFO sqlalchemy.engine.Engine [raw sql] ()
2

In [14]:
def create_heroes():
    with Session(engine) as session:
        team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
        team_z_force = Team(name="Z-Force", headquarters="Sister Margaret’s Bar")

        hero_deadpond = Hero(
            name="Deadpond",
            secret_name="Dive Wilson",
            teams=[team_z_force, team_preventers],
        )
        hero_rusty_man = Hero(
            name="Rusty-Man",
            secret_name="Tommy Sharp",
            age=48,
            teams=[team_preventers],
        )
        hero_spider_boy = Hero(
            name="Spider-Boy", secret_name="Pedro Parqueador", teams=[team_preventers]
        )

        session.add(hero_deadpond)
        session.add(hero_rusty_man)
        session.add(hero_spider_boy)
        session.commit()

        session.refresh(hero_deadpond)
        session.refresh(hero_rusty_man)
        session.refresh(hero_spider_boy)

        cprint(f"Deadpond: {hero_deadpond}", "green")
        cprint(f"Rusty-Man: {hero_rusty_man}", "green")
        cprint(f"\tRusty-Man Teams: {hero_rusty_man.teams}", "green")
        cprint(f"Spider-Boy: {hero_spider_boy}", "green")
        cprint(f"Spider-Boy Teams: {hero_spider_boy.teams}", "green")


create_heroes()

2021-09-08 17:29:35,620 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:29:35,621 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:29:35,621 INFO sqlalchemy.engine.Engine [cached since 327.8s ago] ('Deadpond', 'Dive Wilson', None)
2021-09-08 17:29:35,624 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:29:35,625 INFO sqlalchemy.engine.Engine [cached since 327.8s ago] ('Rusty-Man', 'Tommy Sharp', 48)
2021-09-08 17:29:35,626 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:29:35,627 INFO sqlalchemy.engine.Engine [cached since 327.8s ago] ('Spider-Boy', 'Pedro Parqueador', None)
2021-09-08 17:29:35,628 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:29:35,629 INFO sqlalchemy.engine.Engine [cached since 327.8s ago] ('Z-Force', 'Sister Margaret’s Bar')
2021-09-08 17:29:35

In [15]:
with Session(engine) as session:
    session.execute("delete from hero;")
    session.execute("delete from team;")
    session.execute("delete from heroteamlink;")
    session.commit()

2021-09-08 17:29:38,610 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:29:38,611 INFO sqlalchemy.engine.Engine delete from hero;
2021-09-08 17:29:38,612 INFO sqlalchemy.engine.Engine [cached since 166.4s ago] ()
2021-09-08 17:29:38,615 INFO sqlalchemy.engine.Engine delete from team;
2021-09-08 17:29:38,616 INFO sqlalchemy.engine.Engine [cached since 166.4s ago] ()
2021-09-08 17:29:38,617 INFO sqlalchemy.engine.Engine delete from heroteamlink;
2021-09-08 17:29:38,618 INFO sqlalchemy.engine.Engine [cached since 7.568s ago] ()
2021-09-08 17:29:38,619 INFO sqlalchemy.engine.Engine COMMIT


### [Creating Many-to-Many Relationships](https://sqlmodel.tiangolo.com/tutorial/many-to-many/create-data/)

In [16]:
def create_heroes():
    with Session(engine) as session:
        team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
        team_z_force = Team(name="Z-Force", headquarters="Sister Margaret’s Bar")

        hero_deadpond = Hero(
            name="Deadpond",
            secret_name="Dive Wilson",
            teams=[team_z_force, team_preventers],
        )
        hero_rusty_man = Hero(
            name="Rusty-Man",
            secret_name="Tommy Sharp",
            age=48,
            teams=[team_preventers],
        )
        hero_spider_boy = Hero(
            name="Spider-Boy", secret_name="Pedro Parqueador", teams=[team_preventers]
        )
        session.add(hero_deadpond)
        session.add(hero_rusty_man)
        session.add(hero_spider_boy)
        session.commit()

        session.refresh(hero_deadpond)
        session.refresh(hero_rusty_man)
        session.refresh(hero_spider_boy)

        cprint(f"Deadpond: {hero_deadpond}", "green")
        cprint(f"Deadpond teams: {hero_deadpond.teams}", "green")
        cprint(f"Rusty-Man: {hero_rusty_man}", "green")
        cprint(f"Rusty-Man Teams: {hero_rusty_man.teams}", "green")
        cprint(f"Spider-Boy: {hero_spider_boy}", "green")
        cprint(f"Spider-Boy Teams: {hero_spider_boy.teams}", "green")


create_heroes()

2021-09-08 17:29:40,279 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:29:40,280 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:29:40,281 INFO sqlalchemy.engine.Engine [cached since 332.4s ago] ('Deadpond', 'Dive Wilson', None)
2021-09-08 17:29:40,428 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:29:40,429 INFO sqlalchemy.engine.Engine [cached since 332.6s ago] ('Rusty-Man', 'Tommy Sharp', 48)
2021-09-08 17:29:40,430 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:29:40,432 INFO sqlalchemy.engine.Engine [cached since 332.6s ago] ('Spider-Boy', 'Pedro Parqueador', None)
2021-09-08 17:29:40,433 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:29:40,434 INFO sqlalchemy.engine.Engine [cached since 332.6s ago] ('Z-Force', 'Sister Margaret’s Bar')
2021-09-08 17:29:40

### [Update and Remove Many-to-Many Relationships](https://sqlmodel.tiangolo.com/tutorial/many-to-many/update-remove-relationships/)

In [17]:
def update_heroes():
    with Session(engine) as session:
        # fetch records to update
        hero_spider_boy = session.exec(
            select(Hero).where(Hero.name == "Spider-Boy")
        ).one()
        team_z_force = session.exec(select(Team).where(Team.name == "Z-Force")).one()

        # add hero_spider_boy to team_z_force
        team_z_force.heroes.append(hero_spider_boy)
        session.add(team_z_force)
        session.commit()

        # check objects
        cprint(f"Updated Spider-Boy's Teams: {hero_spider_boy.teams}", "green")
        cprint(f"Z-Force heroes: {team_z_force.heroes}", "green")

        #
        hero_spider_boy.teams.remove(team_z_force)
        session.add(team_z_force)
        session.commit()

        cprint(f"Reverted Z-Force's heroes: {team_z_force.heroes}", "green")
        cprint(f"Reverted Spider-Boy's teams: {hero_spider_boy.teams}", "green")


update_heroes()

2021-09-08 17:29:43,794 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:29:43,796 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero 
WHERE hero.name = ?
2021-09-08 17:29:43,797 INFO sqlalchemy.engine.Engine [no key 0.00090s] ('Spider-Boy',)
2021-09-08 17:29:43,799 INFO sqlalchemy.engine.Engine SELECT team.id, team.name, team.headquarters 
FROM team 
WHERE team.name = ?
2021-09-08 17:29:43,800 INFO sqlalchemy.engine.Engine [no key 0.00072s] ('Z-Force',)
2021-09-08 17:29:43,802 INFO sqlalchemy.engine.Engine SELECT hero.id AS hero_id, hero.name AS hero_name, hero.secret_name AS hero_secret_name, hero.age AS hero_age 
FROM hero, heroteamlink 
WHERE ? = heroteamlink.team_id AND hero.id = heroteamlink.hero_id
2021-09-08 17:29:43,803 INFO sqlalchemy.engine.Engine [generated in 0.00050s] (1,)
2021-09-08 17:29:43,804 INFO sqlalchemy.engine.Engine INSERT INTO heroteamlink (team_id, hero_id) VALUES (?, ?)
2021-09-08 17:29:43,805 INFO sqlal

### [Link Model with Extra Fields](https://sqlmodel.tiangolo.com/tutorial/many-to-many/link-with-extra-fields/)

In [4]:
# RESTART KERNEL, RE-IMPORT AND REMOVE DATABASE.DB
class HeroTeamLink(SQLModel, table=True):
    team_id: Optional[int] = Field(default=None, foreign_key="team.id", primary_key=True)
    hero_id: Optional[int] = Field(default=None, foreign_key="hero.id", primary_key=True)
    is_training: bool = False

    team: "Team" = Relationship(back_populates="hero_links")
    hero: "Hero" = Relationship(back_populates="team_links")


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    headquarters: str

    hero_links: List[HeroTeamLink] = Relationship(back_populates="team")


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

    team_links: List[HeroTeamLink] = Relationship(back_populates="hero")

In [5]:
create_db_and_tables()

2021-09-08 17:31:16,098 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:31:16,099 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("heroteamlink")
2021-09-08 17:31:16,100 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:31:16,101 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("heroteamlink")
2021-09-08 17:31:16,102 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:31:16,103 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("team")
2021-09-08 17:31:16,104 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:31:16,105 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("team")
2021-09-08 17:31:16,106 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:31:16,107 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2021-09-08 17:31:16,108 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-08 17:31:16,109 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2021-09-08 17:31:16,109 INFO sqlalchemy.engine.Engine [raw sql] ()
2

In [6]:
def create_heroes():
    with Session(engine) as session:
        team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
        team_z_force = Team(name="Z-Force", headquarters="Sister Margaret’s Bar")

        hero_deadpond = Hero(
            name="Deadpond",
            secret_name="Dive Wilson",
        )
        hero_rusty_man = Hero(
            name="Rusty-Man",
            secret_name="Tommy Sharp",
            age=48,
        )
        hero_spider_boy = Hero(
            name="Spider-Boy",
            secret_name="Pedro Parqueador",
        )

        deadpond_team_z_link = HeroTeamLink(team=team_z_force, hero=hero_deadpond)
        deadpond_preventers_link = HeroTeamLink(
            team=team_preventers, hero=hero_deadpond, is_training=True
        )

        spider_boy_preventers_link = HeroTeamLink(
            team=team_preventers, hero=hero_spider_boy, is_training=True
        )
        rusty_man_preventers_link = HeroTeamLink(
            team=team_preventers, hero=hero_rusty_man
        )

        session.add(deadpond_team_z_link)
        session.add(deadpond_preventers_link)
        session.add(spider_boy_preventers_link)
        session.add(rusty_man_preventers_link)
        session.commit()

        for link in team_z_force.hero_links:
            cprint(f"Z-Force hero: {link.hero} is training: {link.is_training}", "green")

        for link in team_preventers.hero_links:
            cprint(
                f"Preventers hero: {link.hero} is training: {link.is_training}", "green"
            )


create_heroes()

2021-09-08 17:31:18,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:31:18,834 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:31:18,834 INFO sqlalchemy.engine.Engine [generated in 0.00069s] ('Deadpond', 'Dive Wilson', None)
2021-09-08 17:31:18,838 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:31:18,838 INFO sqlalchemy.engine.Engine [cached since 0.004492s ago] ('Spider-Boy', 'Pedro Parqueador', None)
2021-09-08 17:31:18,839 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?)
2021-09-08 17:31:18,840 INFO sqlalchemy.engine.Engine [cached since 0.005919s ago] ('Rusty-Man', 'Tommy Sharp', 48)
2021-09-08 17:31:18,841 INFO sqlalchemy.engine.Engine INSERT INTO team (name, headquarters) VALUES (?, ?)
2021-09-08 17:31:18,842 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('Z-Force', 'Sister Margaret’s Bar')
2021-09-08 17:31:

In [7]:
def update_heroes():
    with Session(engine) as session:
        hero_spider_boy = session.exec(
            select(Hero).where(Hero.name == "Spider-Boy")
        ).one()
        team_z_force = session.exec(select(Team).where(Team.name == "Z-Force")).one()

        spider_boy_z_force_link = HeroTeamLink(
            team=team_z_force, hero=hero_spider_boy, is_training=True
        )

        team_z_force.hero_links.append(spider_boy_z_force_link)
        session.add(team_z_force)
        session.commit()

        cprint(f"Updated Spider-Boy's Teams: {hero_spider_boy.team_links}", "green")
        cprint(f"Z-Force heroes: {team_z_force.hero_links}", "green")

        for link in hero_spider_boy.team_links:
            if link.team.name == "Preventers":
                link.is_training = False

        session.add(hero_spider_boy)
        session.commit()

        for link in hero_spider_boy.team_links:
            cprint(
                f"Spider-Boy team: {link.team} is training: {link.is_training}", "green"
            )


update_heroes()

2021-09-08 17:31:22,607 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-08 17:31:22,608 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero 
WHERE hero.name = ?
2021-09-08 17:31:22,608 INFO sqlalchemy.engine.Engine [no key 0.00043s] ('Spider-Boy',)
2021-09-08 17:31:22,610 INFO sqlalchemy.engine.Engine SELECT team.id, team.name, team.headquarters 
FROM team 
WHERE team.name = ?
2021-09-08 17:31:22,611 INFO sqlalchemy.engine.Engine [no key 0.00080s] ('Z-Force',)
2021-09-08 17:31:22,614 INFO sqlalchemy.engine.Engine INSERT INTO heroteamlink (team_id, hero_id, is_training) VALUES (?, ?, ?)
2021-09-08 17:31:22,615 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (1, 2, 1)
2021-09-08 17:31:22,711 INFO sqlalchemy.engine.Engine SELECT heroteamlink.team_id AS heroteamlink_team_id, heroteamlink.hero_id AS heroteamlink_hero_id, heroteamlink.is_training AS heroteamlink_is_training 
FROM heroteamlink 
WHERE ? = heroteamlink.team_id
2021-09-08 1